In [1]:
import string
import pandas as pd
import nltk
import ssl

from bs4 import BeautifulSoup

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, hamming_loss
from sklearn.metrics import hamming_loss
from sklearn.model_selection import train_test_split

import catboost as ctb
from catboost.utils import eval_metric


try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
    
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gorinenko/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/gorinenko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Постановка задачи

Задача состоит в автоматической генерации тегов на основе приведенного текста. Количество тегов может быть более одного. Данная проблема в ML называется "multilabel classification". Существует также схожая проблема "multiclass classification", которая не относится к нашей задаче. Разница в том, что "multilabel classification" из множества классов предсказывает более одного класса, в тоже время "multiclass classification" выбирает только один самый вероятный класс.

"Binary сlassification" - предсказание одного класса, при условии, что всего таках классов два.

Предсказывать теги нам потребуется на русском языке, поэтому в качестве обучающего набора данных выбран русский раздел stackoverflow. Данный набор данных можно скачать по ссылке [Stack Exchange Data Dump](https://archive.org/details/stackexchange). Для преобразования xml документов в csv формат используется скрипт convert_to_csv.

# Предобработка текста

Загружаем корпус вопросов со stackoverflow на русском языке. Предобрабатываем колонку с тегами.

In [2]:
file_path = 'data/stackoverflow_posts.csv'
# file_path = 'data/stackexchange_posts.csv'

nrows = 15_000
raw_df = pd.read_csv(file_path)
df = raw_df[~raw_df['Tags'].isna()]

df = df.iloc[:nrows, :]
df.reset_index(inplace=True)
# 20000 - 30 мин
# 5000 - 8 мин
# 100_000 - 3 ч
# Всего - 912090

print(f'samples count {df.shape[0]}')

samples count 15000


In [3]:
df.fillna('', inplace=True)

In [4]:
def parse_tags(value):
    tags = value.replace('<', '').split('>')
    return [tag for tag in tags if tag]

    
df["Tags"] = df["Tags"].apply(lambda x: parse_tags(x))

Предобрабатываем колонку "Body".

1. Приводим к нижнему регистру
2. Разбиваем на токены (слова)
3. Из списка токенов выкидываем стоп-слова и пунктуацию
4. Проводим леммитизацию и стемминг если надо

In [5]:
stop_words = set(stopwords.words('russian'))
punctuation = set(string.punctuation)


class TextPreProcessor:
    def __init__(self, tokenizer, stemmer=None, morph=None):
        self.tokenizer = tokenizer
        self.stemmer = stemmer
        self.morph = morph


    def tokenize(self, text: str):      
        text = text.lower()
      
        doc = BeautifulSoup(text, 'lxml')
        text = doc.text
        
        tokens = self.tokenizer.tokenize(text)
        
        words = [word for word in tokens if word not in stop_words and word not in punctuation]
        
        if self.morph:
            words = [self.morph.parse(word)[0].normal_form for word in words]

        if self.stemmer:
            words = [self.stemmer.stem(word) for word in words]

        return words
  
class NltkTokenizer:    
    def tokenize(self, text: str):      
        return list(word_tokenize(text))

In [6]:
def preprocessor(text):
    if not isinstance(text, str):
          return text
      
    tokenizer = TextPreProcessor(tokenizer=NltkTokenizer())    
    words = tokenizer.tokenize(text)
    return ' '.join(words)
    
df['Body_cleaned'] = df['Body'].apply(lambda x: preprocessor(x))

/var/folders/g5/8xrz2mg176j1yxttghxsl4gc0000gn/T/ipykernel_6594/1889254982.py:15: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  doc = BeautifulSoup(text, 'lxml')


In [7]:
df.head(15)

,index,Id,Title,Body,Tags,Body_cleaned
0,0,1,Как из скрипта на Питоне послать письмо с влож...,<p>Нужен простейший пример посылки письма с вл...,"[python, smtp]",нужен простейший пример посылки письма вложени...
1,1,1,Как из скрипта на Питоне послать письмо с влож...,<p>Нужен простейший пример посылки письма с вл...,"[python, smtp]",нужен простейший пример посылки письма вложени...
2,4,3,Как сохранить и восстановить базу данных Postg...,"<p>Например, имеется пользователь <em>postgres...",[postgresql],например имеется пользователь postgres база ba...
3,5,3,Как сохранить и восстановить базу данных Postg...,"<p>Например, имеется пользователь <em>postgres...",[postgresql],например имеется пользователь postgres база ba...
4,8,5,Как найти файл по имени в папках командой из т...,<p>Какая команда Linux наиболее подходит подоб...,"[linux, файлы]",команда linux наиболее подходит подобного поиска
5,9,5,Как найти файл по имени в папках командой из т...,<p>Какая команда Linux наиболее подходит подоб...,"[linux, файлы]",команда linux наиболее подходит подобного поиска
6,12,7,Как работать с svn через HTTP прокси на Ubuntu?,<p>При попытке извлечения кода svn выдает след...,"[http, svn, прокси]",попытке извлечения кода svn выдает следующую о...
7,13,7,Как работать с svn через HTTP прокси на Ubuntu?,<p>При попытке извлечения кода svn выдает след...,"[http, svn, прокси]",попытке извлечения кода svn выдает следующую о...
8,16,9,Как подсчитать строки в файле из терминала в U...,<p>Какая команда наиболее проста и удобна?</p>,[linux],команда наиболее проста удобна
9,17,9,Как подсчитать строки в файле из терминала в U...,<p>Какая команда наиболее проста и удобна?</p>,[linux],команда наиболее проста удобна


Преобразуем колонку Tags в матрицу с закодированным представлением тегов

In [8]:
multi_label = MultiLabelBinarizer()
Y = multi_label.fit_transform(list(df["Tags"]))

In [9]:
print(f'Классы: \n{list(multi_label.classes_)}')

Классы: 
['.htaccess', '.net', '1с', '2d', '3d', 'access', 'access-control', 'action', 'actionscript', 'actionscript-3', 'active-directory', 'active-perl', 'activex', 'adapter', 'addons', 'ado', 'ado.net', 'adobe', 'adsl', 'agile', 'air', 'ajax', 'android', 'android-adb', 'android-layout', 'android-ndk', 'android-sdk', 'android-sources', 'ant', 'apache', 'apache-flex', 'apache-modules', 'apache-poi', 'apache2.2', 'api', 'applet', 'apt-get', 'aptana', 'arm', 'asp', 'asp.net', 'asp.net-mvc', 'aspx', 'asus', 'at', 'atl', 'autocomplete', 'avr', 'avr-gcc', 'awt', 'backbone.js', 'bash', 'bat', 'batik', 'bbcode', 'beanshell', 'beautiful-soup', 'behaviors', 'bind', 'binding', 'bios', 'blackberry', 'blocks', 'blowfish', 'bluetooth', 'boost', 'boot', 'borland', 'borland-c++', 'box2d', 'brainfuck', 'build', 'c', 'c#', 'c#-faq', 'c++', 'c++-cli', 'c++-faq', 'c++builder', 'c3p0', 'callback', 'canvas', 'captcha', 'carbon', 'case', 'cdt', 'centos', 'cgi', 'chart', 'chm', 'chrome-extension', 'cisco', 

# Feature Engineering

## TfidfVectorizer

Получим векторное представление текстов колонки Body_cleaned с использованием TfidfVectorizer.

TfidfVectorizer формирует матрицу документы-слова, значения ячеек в которой является статистическая мера TF-IDF. 

TF-IDF (от англ. TF — term frequency, IDF — inverse document frequency) — статистическая мера, используемая для оценки важности слова в контексте документа, являющегося частью коллекции документов или корпуса. Вес некоторого слова пропорционален частоте употребления этого слова в документе и обратно пропорционален частоте употребления слова во всех документах коллекции.

Мера TF-IDF часто используется в задачах анализа текстов и информационного поиска, например, как один из критериев релевантности документа поисковому запросу, при расчёте меры близости документов при кластеризации. Также используется в задачах ранжирования и выделение ключевых слов.

Большой вес в TF-IDF получат слова с высокой частотой в пределах конкретного документа и с низкой частотой употреблений в других документах. Чем релевантнее слово к документу, тем больше значение TF-IDF. 

max_df - при построении словаря игнорируются слова, у которых частота появления в документах строго превышает заданный порог

max_features - максимальное количество наиболее часто встречающихся слов в словаре

In [10]:
tfidf= TfidfVectorizer(max_df=0.7, max_features=200)

X_tf = tfidf.fit_transform(df["Body_cleaned"])

In [11]:
tfidf_df =  pd.DataFrame.sparse.from_spmatrix(X_tf, columns=tfidf.get_feature_names_out(), index=df["Body_cleaned"].index)
tfidf_df.head(10)

,10,100,11,20,2011,25,_post,add,and,android,...,типа,то,файл,файла,функции,функция,хочу,число,что,это
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Обучение модели

# Функции потерь

В задаче "multilabel classification" ответами является матрица N x K, где N - количество примеров, а K - количество классов(теги). Для бинарной классификации с несколькими метками существуют две функции потерь - это **MultiLogloss** и **MultiCrossEntropy**. При использовании MultiLogloss предсказание Yik какой-то метки может быть 0 или 1. При использовании MultiCrossEntropy предсказание Yik является вероятностью принадлежности к какой-то метке, и следовательно, значение лежит в интервале от 0 до 1.

## Метрики качества


Обучим различные классификаторы. В качестве метрик в данной модели можно использовать различные метрики. 

Начнем с confusion matrix(матрица ошибок). Допустим, что у нас есть два класса и алгоритм, предсказывающий принадлежность каждого объекта одному из классов, тогда матрица ошибок классификации будет выглядеть следующим образом:
|   | y = 1 | y = 0 |
| --- | --- | --- |
| y_prep = 1 | True Positive (TP) | False Positive (FP) |
| y_prep = 0 | False Negative (FN) | True Negative (TN) |

**Accuracy** - доля правильных ответов. Отношение доли правильно угаданных ответов к общему количеству предсказаний. Эта метрика плохо работает, когда наблюдается дисбаланс классов.

**Precision** - точность. Метрика, вычисленная по формуле **tp / (tp + fp)**. Это способность классификатора не предсказывать положительный класс, если в действительности он является отрицательным. Отвечает на вопрос "Как точно предсказаны правильные метки?". Чем выше точность, тем лучше. Диапазон значений [0, 1].

**Recall** - полнота. Метрика, вычисленная по формуле **tp / (tp + fn)**. Это способность классификатора находить все положительные ответы. Отвечает на вопрос "Какая доля правильных меток предсказана?". Чем выше точность, тем лучше. Диапазон значений [0, 1].

**F1 мера** - среднее гармоническое между precision и recall. Вычисляется по формуле F1 = 2 * (precision * recall) / (precision + recall). F-мера достигает максимума при полноте и точности, равными единице, и близка к нулю, если один из аргументов близок к нулю.

**ROC AUC** - площадь под кривой ошибок. Данная кривая строится в координатах True Positive Rate **tpr = tp / (tp + fn)** и False Positive Rate **fpr = fp / (fp + tn)** от [0,0] до [1, 1]. tpr - это полнота, а fpr показывает, какую долю из объектов negative класса алгоритм предсказал неверно, чем меньше это значение, тем лучше. Следовательно, когда классификатор не делает ошибок (fpr = 0, tpr = 1) площадь под кривой равна единице. Когда классификатор случайно выдает вероятности классов, AUC-ROC будет стремиться к 0.5. Чем выше значение, тем лучше. Диапазон значений [0, 1]. Кроме этого, важной является крутизна кривой т.к желательно максимизировать tpr, минимизируя fpr, т.е. кривая в должна стремиться к точке [0, 1].

В задаче бинарной классификации с несколькими метками метрики Precision, Recall, F1, ROC AUC вычисляются для каждой метки-тега, а затем каким-то образом агрегируются или анализируются для каждой метки в отдельности. Scikit learn имеет параметер average для precision_score, recall_score и f1_score, roc_auc_score. Precision и recall не зависят, в отличие от accuracy, от соотношения классов и потому применимы в условиях несбалансированных выборок.

**HammingLoss** - метрика, определяющая долю меток, которые были предсказаны неправильно, по сути - это средняя точность по классу, вычтенная из единицы. Чем меньше значение метрики, тем лучше. Диапазон значений [0, 1]. Данную метрику можно использовать в eval_metric для оценки модели и механизма ранней остановки обучения.

## Модель

В качестве модели используем классификатор CatBoostClassifier.

In [12]:
random_state=42

X_train, X_test, y_train, y_test = train_test_split(X_tf, Y, test_size=0.1, random_state=random_state, shuffle=True)
X_train = X_train.toarray()
X_test = X_test.toarray()

In [13]:
from catboost import Pool


device = 'cpu'
train_params = {
    'random_state': random_state, 
    'eval_metric': 'HammingLoss', 
    'task_type': device.upper(), 
    'metric_period': 10, 
    'verbose': 50,
    'iterations': 100,
    'allow_const_label': True,
    # 'logging_level': 'Silent',  
    'loss_function': 'MultiCrossEntropy',
    # 'loss_function': 'MultiLogloss',
    'class_names': multi_label.classes_
}

In [14]:
model = ctb.CatBoostClassifier(**train_params)
model.fit(Pool(X_train, y_train), eval_set=Pool(X_test, y_test), plot=True, use_best_model = True)                

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.164097
0:	learn: 0.0021657	test: 0.0021685	best: 0.0021685 (0)	total: 15.9s	remaining: 26m 9s
50:	learn: 0.0018929	test: 0.0019364	best: 0.0019364 (50)	total: 13m 54s	remaining: 13m 21s
99:	learn: 0.0017495	test: 0.0018670	best: 0.0018670 (99)	total: 27m 29s	remaining: 0us

bestTest = 0.001866962306
bestIteration = 99



## Оценка модели

Оценим метрики scikit learn

In [15]:
preds = model.predict(X_test)
probs = model.predict_proba(X_test)

# Делается предсказание для каждого класса
assert preds.shape[1] == multi_label.classes_.shape[0]
assert probs.shape[1] == multi_label.classes_.shape[0]

In [16]:
preds.shape

(1500, 902)

In [17]:
accuracy = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds, average=None, zero_division=0)
recall = recall_score(y_test, preds, average=None, zero_division=0)
f1 = f1_score(y_test, preds, average=None, zero_division=0)
auc = roc_auc_score(y_test, probs[:,1].reshape(-1, 1), average=None)
hamming = hamming_loss(y_test, preds)


print(f'accuracy: {accuracy}\n')
print(f'precision: {list(precision)}\n')
print(f'recall: {list(recall)}\n')
print(f'f1: {list(f1)}\n')
print(f'auc: {auc}\n')
print(f'hamming: {hamming}\n')

accuracy: 0.12

precision: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6666666666666666, 1.0, 0.0, 0.8333333333333334, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.9130434782608695, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7692307692307693, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

Сравним полученные метрики с метриками пакета catboost.

Средняя точность по всем классам:

In [18]:
accuracy = eval_metric(y_test, preds, 'Accuracy')[0]
print(f'Accuracy: {accuracy}')

Accuracy: 0.12


Точность предсказания каждого класса в отдельности:

In [19]:
accuracy_per_class = eval_metric(y_test, preds, 'Accuracy:type=PerClass')
for cls, value in zip(multi_label.classes_, accuracy_per_class):
    print(f'Accuracy for class {cls}: {value}')

Accuracy for class .htaccess: 0.994
Accuracy for class .net: 0.984
Accuracy for class 1с: 0.9973333333333333
Accuracy for class 2d: 1.0
Accuracy for class 3d: 0.998
Accuracy for class access: 0.9993333333333333
Accuracy for class access-control: 0.9993333333333333
Accuracy for class action: 1.0
Accuracy for class actionscript: 0.9946666666666667
Accuracy for class actionscript-3: 0.9966666666666667
Accuracy for class active-directory: 1.0
Accuracy for class active-perl: 0.9993333333333333
Accuracy for class activex: 0.9993333333333333
Accuracy for class adapter: 1.0
Accuracy for class addons: 1.0
Accuracy for class ado: 1.0
Accuracy for class ado.net: 0.9993333333333333
Accuracy for class adobe: 0.9993333333333333
Accuracy for class adsl: 0.9993333333333333
Accuracy for class agile: 1.0
Accuracy for class air: 1.0
Accuracy for class ajax: 0.992
Accuracy for class android: 0.978
Accuracy for class android-adb: 1.0
Accuracy for class android-layout: 0.9973333333333333
Accuracy for class 

Метрики Precision, Recall и F1 также считаются для каждого класса в отдельности:

In [20]:
print('Precision')

values = eval_metric(y_test, preds, 'Precision')
for cls, value in zip(multi_label.classes_, values):
    print(f'class={cls}: {value:.4f}')


Precision
class=.htaccess: 1.0000
class=.net: 1.0000
class=1с: 1.0000
class=2d: 1.0000
class=3d: 1.0000
class=access: 1.0000
class=access-control: 1.0000
class=action: 1.0000
class=actionscript: 1.0000
class=actionscript-3: 1.0000
class=active-directory: 1.0000
class=active-perl: 1.0000
class=activex: 1.0000
class=adapter: 1.0000
class=addons: 1.0000
class=ado: 1.0000
class=ado.net: 1.0000
class=adobe: 1.0000
class=adsl: 1.0000
class=agile: 1.0000
class=air: 1.0000
class=ajax: 1.0000
class=android: 1.0000
class=android-adb: 1.0000
class=android-layout: 1.0000
class=android-ndk: 1.0000
class=android-sdk: 1.0000
class=android-sources: 1.0000
class=ant: 1.0000
class=apache: 1.0000
class=apache-flex: 1.0000
class=apache-modules: 1.0000
class=apache-poi: 1.0000
class=apache2.2: 1.0000
class=api: 1.0000
class=applet: 1.0000
class=apt-get: 1.0000
class=aptana: 1.0000
class=arm: 1.0000
class=asp: 1.0000
class=asp.net: 1.0000
class=asp.net-mvc: 1.0000
class=aspx: 1.0000
class=asus: 1.0000
class

In [21]:
print('Recall')

values = eval_metric(y_test, preds, 'Recall')
for cls, value in zip(multi_label.classes_, values):
    print(f'class={cls}: {value:.4f}')

Recall
class=.htaccess: 0.0000
class=.net: 0.0000
class=1с: 0.0000
class=2d: 1.0000
class=3d: 0.0000
class=access: 0.0000
class=access-control: 0.0000
class=action: 1.0000
class=actionscript: 0.0000
class=actionscript-3: 0.0000
class=active-directory: 1.0000
class=active-perl: 0.0000
class=activex: 0.0000
class=adapter: 1.0000
class=addons: 1.0000
class=ado: 1.0000
class=ado.net: 0.0000
class=adobe: 0.0000
class=adsl: 0.0000
class=agile: 1.0000
class=air: 1.0000
class=ajax: 0.0000
class=android: 0.3529
class=android-adb: 1.0000
class=android-layout: 0.0000
class=android-ndk: 0.0000
class=android-sdk: 0.0000
class=android-sources: 0.0000
class=ant: 0.0000
class=apache: 0.0000
class=apache-flex: 0.0000
class=apache-modules: 0.0000
class=apache-poi: 1.0000
class=apache2.2: 1.0000
class=api: 1.0000
class=applet: 0.0000
class=apt-get: 1.0000
class=aptana: 1.0000
class=arm: 0.0000
class=asp: 1.0000
class=asp.net: 0.1667
class=asp.net-mvc: 0.0000
class=aspx: 1.0000
class=asus: 1.0000
class=at

In [22]:
print('F1')

values = eval_metric(y_test, preds, 'F1')
for cls, value in zip(multi_label.classes_, values):
    print(f'class={cls}: {value:.4f}')

F1
class=.htaccess: 0.0000
class=.net: 0.0000
class=1с: 0.0000
class=2d: 1.0000
class=3d: 0.0000
class=access: 0.0000
class=access-control: 0.0000
class=action: 1.0000
class=actionscript: 0.0000
class=actionscript-3: 0.0000
class=active-directory: 1.0000
class=active-perl: 0.0000
class=activex: 0.0000
class=adapter: 1.0000
class=addons: 1.0000
class=ado: 1.0000
class=ado.net: 0.0000
class=adobe: 0.0000
class=adsl: 0.0000
class=agile: 1.0000
class=air: 1.0000
class=ajax: 0.0000
class=android: 0.5217
class=android-adb: 1.0000
class=android-layout: 0.0000
class=android-ndk: 0.0000
class=android-sdk: 0.0000
class=android-sources: 0.0000
class=ant: 0.0000
class=apache: 0.0000
class=apache-flex: 0.0000
class=apache-modules: 0.0000
class=apache-poi: 1.0000
class=apache2.2: 1.0000
class=api: 1.0000
class=applet: 0.0000
class=apt-get: 1.0000
class=aptana: 1.0000
class=arm: 0.0000
class=asp: 1.0000
class=asp.net: 0.2857
class=asp.net-mvc: 0.0000
class=aspx: 1.0000
class=asus: 1.0000
class=at: 0.

HammingLoss, разность единицы и средней точности предсказания каждого класса в отдельности:

In [23]:
hamming = eval_metric(y_test, preds, 'HammingLoss')[0]
print(f'HammingLoss: {hamming:.4f}')
mean_accuracy_per_class = sum(accuracy_per_class) / len(accuracy_per_class)
print(f'MeanAccuracyPerClass: {mean_accuracy_per_class:.4f}')
print(f'HammingLoss + MeanAccuracyPerClass = {hamming + mean_accuracy_per_class}')


HammingLoss: 0.0019
MeanAccuracyPerClass: 0.9981
HammingLoss + MeanAccuracyPerClass = 0.9999999999999962


## Предсказания модели

In [24]:
valid_df = df[df['Tags'].isna()]

In [42]:
import random

valid_df = raw_df[~raw_df['Tags'].isna()]
valid_df.reset_index(inplace=True)


row_num = random.randint(0, valid_df['Body'].shape[0])
text = valid_df['Body'][row_num]

print(f'Текст: \n{text}\n')

X_tf_val = tfidf.transform([preprocessor(text)])

preds = model.predict(X_tf_val)

labels = multi_label.inverse_transform(preds)
print(f'Предсказанные теги: \n{labels}\n')

Текст: 
<p>Помогите получить Alt текст для изображения при использовании своего шаблона <em>createFromHtml</em></p>

<pre><code>CKEDITOR.on('dialogDefinition', function(ev) {
    var dialogName = ev.data.name;
    var dialogDefinition = ev.data.definition;

    var editor = ev.editor;
    if (dialogName == 'image') {
       dialogDefinition.onOk = function(e) {
          var imageSrcUrl = e.sender.originalElement.$.src;
          var imgHtml = CKEDITOR.dom.element.createFromHtml("&lt;div class='img'&gt;&lt;a href="+ imageSrcUrl + "&gt;&lt;img class='img-responsive' src=" + imageSrcUrl + " alt='' /&gt;&lt;/a&gt;&lt;/div&gt;");
          editor.insertElement(imgHtml);
       };
    }
});
</code></pre>

<p>При загрузке изображения, прописываю ALT, но редактор не сохраняет его.
Я так понимаю что нужно получить его аналогично imageSrcUrl, но не могу понять как. Пробовал прописать txtAlt, не помогает.</p>

<p>Спасибо.</p>


Предсказанные теги: 
[('javascript', 'jquery')]



In [46]:
model.save_model(f"data/tf_idf_{nrows}")

# Выводы

При использовании Tfidf для построения эмбеддингов слов, векторы имеют какой-то смысл, однако не учитывается контекст вокруг слова. Также к недостаткам можно отнести сложность построения векторных представлений слов и словаря, который к тому же имеет фиксированный размер. При изменении корпуса текста, векторы следует пересчитывать занова для всех документов.